In [1]:
import numpy as np
import cv2

import matplotlib.pyplot as plt

%matplotlib inline

In [10]:
def extract_video_first_frame(path, use_cv2=True):
    
    cap = cv2.VideoCapture(path)
    
    ret, frame = cap.read()
    
    if use_cv2:
        cv2.imshow('First Frame', frame)

        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
    else:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        plt.imshow(frame)
        plt.axis('off')

In [12]:
PATH = './data/videos/rcpc_bimbambum.mp4'
extract_video_first_frame(PATH)